# 在自訂 RunnableLambda 中使用模型的多種方法

本範例展示如何在自訂處理步驟中調用 LLM 模型，提供四種不同的實現方式。

## 學習重點
- 在 RunnableLambda 中使用模型（閉包方式）
- 使用 RunnableParallel 並行處理多個模型調用
- 串聯多個模型調用
- RunnableLambda 返回 Prompt 再串接模型


## 準備工作：導入套件和建立模型


In [ ]:
# 導入必要的套件
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableParallel
from langchain_google_genai import ChatGoogleGenerativeAI

# 載入環境變數
load_dotenv()

# 建立 Gemini 模型
model = ChatGoogleGenerativeAI(model="gemini-flash-2.5")

print("✅ 模型已建立")


## 方法 1️⃣：在 RunnableLambda 中調用模型（閉包方式）

這種方法通過閉包捕獲外部的模型實例，在自訂函數中直接調用模型。

### ✨ 特點
- 在自訂函數內部直接調用模型
- 可以完全控制處理邏輯
- 適合需要複雜條件判斷的場景


In [ ]:
# 定義主提示模板
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一個專業的客服代表。"),
    ("human", "請回覆這封客戶郵件：\n\n{email_content}")
])

# 方法 1：在 RunnableLambda 中直接調用模型
def format_with_quality_check(reply):
    """格式化郵件並使用模型進行品質檢查"""
    
    # 先格式化郵件
    formatted_reply = f"""
親愛的客戶，

{reply}

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
客服團隊
"""
    
    # 使用模型進行品質檢查
    quality_check_prompt = ChatPromptTemplate.from_messages([
        ("system", "你是一個郵件品質審查專家。請評估這封郵件的專業度、友善度和完整度，並給出1-10分的評分和簡短評語。"),
        ("human", "請評估這封郵件：\n\n{email}")
    ])
    
    # 調用模型進行品質檢查
    quality_result = quality_check_prompt.format_prompt(email=formatted_reply)
    quality_score = model.invoke(quality_result.to_string())
    
    # 組合最終結果
    final_result = f"""
{'='*60}
📧 格式化的郵件回覆：
{'='*60}
{formatted_reply}

{'='*60}
📊 AI 品質檢查結果：
{'='*60}
{quality_score}
"""
    return final_result

# 建立包含模型調用的自訂處理步驟
format_and_check = RunnableLambda(format_with_quality_check)

# 建立完整的鏈
chain_method1 = prompt_template | model | StrOutputParser() | format_and_check

print("✅ 方法 1 的鏈已建立完成！")
print("流程：主 Prompt → 主模型 → 格式化 + 品質檢查模型 → 最終結果")


In [ ]:
# 測試方法 1
customer_email = """
您好，

我最近購買了貴公司的產品，但是發現包裝有損壞。
請問可以退貨或換貨嗎？

王小明
"""

print("\n" + "="*60)
print("方法 1 測試結果：在 RunnableLambda 中調用模型")
print("="*60)
result1 = chain_method1.invoke({"email_content": customer_email})
print(result1)


## 方法 2️⃣：使用 RunnableParallel 並行處理

這種方法適合需要同時調用多個模型進行不同分析的場景。

### ✨ 特點
- 可同時執行多個模型調用
- 提高處理效率
- 適合需要多角度分析的場景


In [ ]:
# 定義格式化函數（不含模型調用）
def format_email(reply):
    """格式化郵件回覆"""
    return f"""
親愛的客戶，

{reply}

感謝您的來信。

此致
客服團隊
"""

# 建立品質檢查鏈（包含模型）
quality_check_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是郵件品質審查專家。請評估這封郵件的專業度，給出1-10分評分和簡短評語。"),
    ("human", "評估這封郵件：\n\n{email}")
])
quality_check_chain = quality_check_prompt | model | StrOutputParser()

# 建立語氣分析鏈（包含模型）
tone_analysis_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是語氣分析專家。請分析這封郵件的語氣是否友善、專業。"),
    ("human", "分析這封郵件的語氣：\n\n{email}")
])
tone_analysis_chain = tone_analysis_prompt | model | StrOutputParser()

# 使用 RunnableParallel 並行執行格式化和兩個模型分析
parallel_chain = RunnableParallel(
    formatted_email=RunnableLambda(format_email),
    quality_check=RunnableLambda(lambda x: {"email": format_email(x)}) | quality_check_chain,
    tone_analysis=RunnableLambda(lambda x: {"email": format_email(x)}) | tone_analysis_chain
)

# 合併結果的函數
def combine_results(parallel_results):
    """合併並行處理的結果"""
    return f"""
{'='*60}
📧 格式化的郵件回覆：
{'='*60}
{parallel_results['formatted_email']}

{'='*60}
📊 AI 品質檢查結果：
{'='*60}
{parallel_results['quality_check']}

{'='*60}
🎭 AI 語氣分析結果：
{'='*60}
{parallel_results['tone_analysis']}
"""

# 建立完整的鏈
chain_method2 = (
    prompt_template 
    | model 
    | StrOutputParser() 
    | parallel_chain 
    | RunnableLambda(combine_results)
)

print("✅ 方法 2 的鏈已建立完成！")
print("流程：主 Prompt → 主模型 → 並行處理（格式化 + 品質檢查模型 + 語氣分析模型）→ 合併結果")


In [ ]:
# 測試方法 2
print("\n" + "="*60)
print("方法 2 測試結果：使用 RunnableParallel 並行處理")
print("="*60)
result2 = chain_method2.invoke({"email_content": customer_email})
print(result2)


## 方法 3️⃣：串聯多個模型調用

這種方法最簡單直接，適合需要順序執行多個模型的場景。

### ✨ 特點
- 結構清晰，易於理解
- 每個模型處理一個特定任務
- 適合需要逐步改進的場景


In [ ]:
# 第一步：生成郵件回覆
reply_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是專業的客服代表。"),
    ("human", "請回覆這封客戶郵件：\n\n{email_content}")
])

# 第二步：格式化郵件
format_prompt = ChatPromptTemplate.from_messages([
    ("system", "請將以下郵件回覆格式化，加上適當的稱呼和結尾。"),
    ("human", "郵件內容：\n\n{reply}")
])

# 第三步：品質檢查和改進建議
improve_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是郵件品質專家。請評估這封郵件並提供改進建議。"),
    ("human", "評估並改進這封郵件：\n\n{formatted_reply}")
])

# 串聯三個模型調用
chain_method3 = (
    reply_prompt 
    | model 
    | StrOutputParser() 
    | (lambda x: {"reply": x}) 
    | format_prompt 
    | model 
    | StrOutputParser() 
    | (lambda x: {"formatted_reply": x}) 
    | improve_prompt 
    | model 
    | StrOutputParser()
)

print("✅ 方法 3 的鏈已建立完成！")
print("流程：生成回覆模型 → 格式化模型 → 改進建議模型")


In [ ]:
# 測試方法 3
print("\n" + "="*60)
print("方法 3 測試結果：串聯多個模型調用")
print("="*60)
result3 = chain_method3.invoke({"email_content": customer_email})
print(result3)


## 方法 4️⃣：RunnableLambda 返回 Prompt + 模型串接（推薦！）

這種方法讓 RunnableLambda 只負責準備 Prompt，然後串接模型調用。

### ✨ 特點
- **最佳實踐**：結合靈活性和可讀性
- 結構清晰，易於維護
- 可以根據內容動態調整處理邏輯


In [ ]:
# 自訂函數：根據郵件內容準備不同的檢查 prompt
def prepare_quality_check_prompt(reply):
    """根據回覆內容準備品質檢查 prompt"""
    formatted = f"""
親愛的客戶，

{reply}

此致
客服團隊
"""
    
    # 簡單的內容分析，決定檢查重點
    if "退貨" in reply or "換貨" in reply:
        check_focus = "請特別注意退換貨政策的說明是否清楚。"
    elif "抱歉" in reply or "歉意" in reply:
        check_focus = "請特別注意道歉的誠意和補償措施。"
    else:
        check_focus = "請評估整體的專業度和友善度。"
    
    return {
        "email": formatted,
        "focus": check_focus
    }

# 品質檢查 prompt（接收自訂函數的輸出）
dynamic_quality_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是郵件品質審查專家。{focus}"),
    ("human", "評估這封郵件：\n\n{email}")
])

# 建立完整的鏈
chain_method4 = (
    prompt_template 
    | model 
    | StrOutputParser() 
    | RunnableLambda(prepare_quality_check_prompt) 
    | dynamic_quality_prompt 
    | model 
    | StrOutputParser()
)

print("✅ 方法 4 的鏈已建立完成！")
print("流程：主 Prompt → 主模型 → 動態準備檢查 Prompt → 品質檢查模型")


In [ ]:
# 測試方法 4
print("\n" + "="*60)
print("方法 4 測試結果：RunnableLambda 返回 Prompt + 模型串接")
print("="*60)
result4 = chain_method4.invoke({"email_content": customer_email})
print(result4)


## 💡 四種方法的比較總結

| 方法 | 優點 | 缺點 | 適用場景 |
|------|------|------|----------|
| **方法 1：閉包調用** | 靈活度高，可以在函數內部完全控制邏輯 | 代碼可讀性較差，不易追蹤鏈的流程 | 需要複雜條件判斷或多步處理 |
| **方法 2：並行處理** | 可同時執行多個模型，提高效率 | 需要合併結果，增加複雜度 | 需要從多個角度同時分析 |
| **方法 3：串聯調用** | 結構清晰，易於理解和維護 | 執行時間較長（順序執行） | 需要多步驟逐步改進的場景 |
| **方法 4：動態 Prompt** | 結合靈活性和可讀性，最佳實踐 | 需要仔細設計 Prompt 格式 | 需要根據內容動態調整處理邏輯 |

## 🎯 使用建議

- 🥇 **推薦使用方法 4**：結構清晰、易於維護、靈活度高
- 🥈 **次選方法 2**：需要並行分析時的最佳選擇
- 🥉 **方法 3**：簡單直接，適合學習和快速原型
- ⚠️ **謹慎使用方法 1**：除非確實需要複雜的內部邏輯

## 🔑 關鍵要點

1. **模型實例可以在閉包中使用**：RunnableLambda 函數可以訪問外部的 `model` 變數
2. **RunnableParallel 提升效率**：當需要多個獨立的模型分析時，使用並行處理
3. **串聯模型很簡單**：使用 `|` 運算符就能輕鬆串接多個模型調用
4. **動態準備 Prompt 最靈活**：在 RunnableLambda 中準備 dict 格式的數據，傳給下一個 Prompt Template

## 📚 延伸學習

- 探索 `RunnableBranch`：根據條件選擇不同的處理路徑
- 學習 `RunnablePassthrough`：保留原始輸入數據
- 研究 `RunnableMap`：將輸入映射到多個輸出
